In [ ]:
# default_exp diffquant_utils

# Utility Functions

## Array/Dict reformatting and transformation functions

In [ ]:
#export
def get_condpairname(condpair):
    return f"{condpair[0]}_VS_{condpair[1]}"

In [ ]:
#export
def get_middle_elem(sorted_list):
    nvals = len(sorted_list)
    if nvals==1:
        return sorted_list[0]
    middle_idx = nvals//2
    if nvals%2==1:
        return sorted_list[middle_idx]
    return 0.5* (sorted_list[middle_idx] + sorted_list[middle_idx-1])

In [ ]:
#export
import numpy as np
def get_nonna_array(array_w_nas):
    res = []
    isnan_arr = np.isnan(array_w_nas)

    for idx in range(len(array_w_nas)):
        sub_res = []
        sub_array = array_w_nas[idx]
        na_array = isnan_arr[idx]
        for idx2 in range(len(sub_array)):
            if not na_array[idx2]:
               sub_res.append(sub_array[idx2])
        res.append(np.array(sub_res))
    return np.array(res)

In [ ]:
#export
import numpy as np
def get_non_nas_from_pd_df(df):
    return {
        pep_name: sub_vals[~np.isnan(sub_vals)] for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
import numpy as np
def get_ionints_from_pd_df(df):
    return {
        pep_name: sub_vals for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
def invert_dictionary(my_map):
    inv_map = {}
    for k, v in my_map.items():
        inv_map[v] = inv_map.get(v, []) + [k]
    return inv_map

In [ ]:
#export
from scipy.stats import norm

def get_z_from_p_empirical(p_emp,p2z):
    p_rounded = np.format_float_scientific(p_emp, 1)
    if p_rounded in p2z:
        return p2z.get(p_rounded)
    z = norm.ppf(float(p_rounded))
    p2z[p_rounded] = z
    return z

In [ ]:
#export
import anytree
def get_levelnodes_from_nodeslist(nodeslist, level):
    levelnodes = []
    for node in nodeslist:
        precursors = anytree.findall(node, filter_= lambda x : (x.type == level))
        levelnodes.extend(precursors)
    return levelnodes

## Documentation functions

In [ ]:
#export
import logging
import os
import sys


def set_logger(
    *,log_path = None,
    log_file_name="",
    stream: bool = True,
    log_level: int = logging.INFO,
    overwrite: bool = False,
) -> str:
    """Set the log stream and file.
    All previously set handlers will be disabled with this command.
    Parameters
    ----------
    log_file_name : str, None
        The file name to where the log is written.
        Folders are automatically created if needed.
        This is relative to the current path. When an empty string is provided,
        a log is written to the AlphaTims "logs" folder with the name
        "log_yymmddhhmmss" (reversed timestamp year to seconds).
        If None, no log file is saved.
        Default is "".
    stream : bool
        If False, no log data is sent to stream.
        If True, all logging can be tracked with stdout stream.
        Default is True.
    log_level : int
        The logging level. Usable values are defined in Python's "logging"
        module.
        Default is logging.INFO.
    overwrite : bool
        If True, overwrite the log_file if one exists.
        If False, append to this log file.
        Default is False.
    Returns
    -------
    : str
        The file name to where the log is written.
    """
    import time
    global PROGRESS_CALLBACK
    log_path = os.path.join(BASE_PATH, "logs")
    root = logging.getLogger()
    formatter = logging.Formatter(
        '%(asctime)s> %(message)s', "%Y-%m-%d %H:%M:%S"
    )
    root.setLevel(log_level)
    while root.hasHandlers():
        root.removeHandler(root.handlers[0])
    if stream:
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_handler.setLevel(log_level)
        stream_handler.setFormatter(formatter)
        root.addHandler(stream_handler)
    if log_file_name is not None:
        if log_file_name == "":
            if not os.path.exists(log_path):
                os.makedirs(log_path)
            log_file_name = log_path
        log_file_name = os.path.abspath(log_file_name)
        if os.path.isdir(log_file_name):
            current_time = time.localtime()
            current_time = "".join(
                [
                    f'{current_time.tm_year:04}',
                    f'{current_time.tm_mon:02}',
                    f'{current_time.tm_mday:02}',
                    f'{current_time.tm_hour:02}',
                    f'{current_time.tm_min:02}',
                    f'{current_time.tm_sec:02}',
                ]
            )
            log_file_name = os.path.join(
                log_file_name,
                f"log_{current_time}.txt"
            )
        directory = os.path.dirname(log_file_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
        if overwrite:
            file_handler = logging.FileHandler(log_file_name, mode="w")
        else:
            file_handler = logging.FileHandler(log_file_name, mode="a")
        file_handler.setLevel(log_level)
        file_handler.setFormatter(formatter)
        root.addHandler(file_handler)
    return log_file_name

In [ ]:
#export
import yaml
import os.path

def load_method_parameters(results_dir):
    params_file = f"{results_dir}/aq_parameters.yaml"
    return load_config(params_file)

def store_method_parameters(local_vars, results_dir):
    method_params = {x : local_vars[x] for x in local_vars.keys() if (("_df" not in x) and ('condpair' not in x))}
    params_file = f"{results_dir}/aq_parameters.yaml"
    if os.path.exists(params_file):
        previous_params = load_method_parameters(results_dir)
        method_params.update(previous_params)
    if not os.path.exists(f"{results_dir}/"):
        os.makedirs(f"{results_dir}/")
    with open(params_file, 'w') as outfile:
        yaml.dump(method_params, outfile, default_flow_style=False)

# Input Parsers
The Alphaquant pipeline is run using a generic wide-table input format, as specified in the documentation. The following parsers convert long format tables as provided e.g. by Spectronaut or DIA-NN into this generic format. The configuration for the parsers is set by a yaml file.

## Convert long format to wide format

### Parse .yaml file
The relevant parameters for reading and reformatting the long table are stored in the "intable_config.yaml" file. The functions below are for reading and reformating the config info

In [ ]:
#export
import yaml
import itertools

def get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict):
    filtcols = []
    for filtconf in filter_dict.values():
        filtcols.append(filtconf.get('param'))
    relevant_cols = protein_cols + ion_cols + [sample_ID] + [quant_ID] + filtcols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols


def get_relevant_columns_config_dict(config_typedict):
    filtcols = []
    dict_ioncols = []
    for filtconf in config_typedict.get('filters', {}).values():
        filtcols.append(filtconf.get('param'))

    if 'ion_hierarchy' in config_typedict.keys():
        for headr in config_typedict.get('ion_hierarchy').values():
            ioncols = list(itertools.chain.from_iterable(headr.get("mapping").values()))
            dict_ioncols.extend(ioncols)
    quantID = config_typedict.get("quant_ID")
    if type(quantID) ==type("string"):
        quant_ids = [config_typedict.get("quant_ID")]
    elif quantID == None:
        quant_ids = []
    else:
        quant_ids = list(config_typedict.get("quant_ID").values())
    relevant_cols = config_typedict.get("protein_cols") + config_typedict.get("ion_cols", []) + [config_typedict.get("sample_ID")] + quant_ids + filtcols + dict_ioncols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols


def get_config_columns(config_dict):
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    sample_ID = config_dict.get("sample_ID")
    quant_ID = config_dict.get("quant_ID")
    filter_dict = config_dict.get("filters", {})
    relevant_cols = get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict)
    return relevant_cols, protein_cols, ion_cols, sample_ID, quant_ID, filter_dict


def load_config(config_yaml):
    stream = open(config_yaml, 'r')
    config_all = yaml.safe_load(stream)
    return config_all

def get_type2relevant_cols(config_all):
    type2relcols = {}
    for type in config_all.keys():
        config_typedict = config_all.get(type)
        relevant_cols = get_relevant_columns_config_dict(config_typedict)
        type2relcols[type] = relevant_cols
    return type2relcols

### Filter and reformat

In [ ]:
#export

def filter_input(filter_dict, input):
    for filtname,filterconf in filter_dict.items():
        param = filterconf.get('param')
        comparator = filterconf.get('comparator')
        value = filterconf.get('value')

        if comparator not in [">",">=", "<", "<=", "==", "!="]:
            raise TypeError(f"cannot identify the filter comparator of {filtname} given in the longtable config yaml!")

        if comparator=="==":
            input = input[input[param] ==value]
            continue
        try:
            input = input.astype({f"{param}" : "float"})
        except:
            pass

        if comparator==">":
            input = input[input[param].astype(type(value)) >value]

        if comparator==">=":
            input = input[input[param].astype(type(value)) >=value]

        if comparator=="<":
            input = input[input[param].astype(type(value)) <value]

        if comparator=="<=":
            input = input[input[param].astype(type(value)) <=value]

        if comparator=="!=":
            input = input[input[param].astype(type(value)) !=value]

    return input

In [ ]:
#export
def merge_protein_and_ion_cols(input_df, config_dict):
    protein_cols =  config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)
    input_df['ion'] = input_df.loc[:, ion_cols].astype('string').sum(axis=1)
    input_df = input_df.rename(columns = {config_dict.get('quant_ID') : "quant_val"})
    return input_df

In [ ]:
#export
import copy
def merge_protein_cols_and_ion_dict(input_df, config_dict):
    """[summary]

    Args:
        input_df ([pandas dataframe]): longtable containing peptide intensity data
        confid_dict ([dict[String[]]]): nested dict containing the parse information. derived from yaml file

    Returns:
        pandas dataframe: longtable with newly assigned "protein" and "ion" columns
    """
    protein_cols = config_dict.get("protein_cols")
    ion_hierarchy = config_dict.get("ion_hierarchy")
    splitcol2sep = config_dict.get('split_cols')
    quant_id_dict = config_dict.get('quant_ID')

    ion_dfs = []
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)

    input_df = input_df.drop(columns = [x for x in protein_cols if x!='protein'])
    for hierarchy_type in ion_hierarchy.keys():
        df_subset = input_df.copy()
        ion_hierarchy_local = ion_hierarchy.get(hierarchy_type).get("order")
        ion_headers_merged, ion_headers_grouped = get_ionname_columns(ion_hierarchy.get(hierarchy_type).get("mapping"), ion_hierarchy_local) #ion headers merged is just a helper to select all relevant rows, ionheaders grouped contains the sets of ionstrings to be merged into a list eg [[SEQ, MOD], [CH]]
        quant_columns = get_quantitative_columns(df_subset, hierarchy_type, config_dict, ion_headers_merged)
        headers = list(set(ion_headers_merged + quant_columns + ['protein']))
        if "sample_ID" in config_dict.keys():
            headers+=[config_dict.get("sample_ID")]
        df_subset = df_subset[headers].drop_duplicates()

        if splitcol2sep is not None:
            if quant_columns[0] in splitcol2sep.keys(): #in the case that quantitative values are stored grouped in one column (e.g. msiso1,msiso2,msiso3, etc.), reformat accordingly
                df_subset = split_extend_df(df_subset, splitcol2sep)
            ion_headers_grouped = adapt_headers_on_extended_df(ion_headers_grouped, splitcol2sep)

        #df_subset = df_subset.set_index(quant_columns)

        df_subset = add_merged_ionnames(df_subset, ion_hierarchy_local, ion_headers_grouped, quant_id_dict, hierarchy_type)
        ion_dfs.append(df_subset)
    input_df = pd.concat(ion_dfs, ignore_index=True)
    return input_df


def get_quantitative_columns(input_df, hierarchy_type, config_dict, ion_headers_merged):
    naming_columns = ion_headers_merged + ['protein']
    if config_dict.get("format") == 'longtable':
        quantcol = config_dict.get("quant_ID").get(hierarchy_type)
        return [quantcol]

    if config_dict.get("format") == 'widetable':
        quantcolumn_candidates = [x for x in input_df.columns if x not in naming_columns]
        if "quant_prefix" in config_dict.keys():
            return [x for x in quantcolumn_candidates if x.startswith(config_dict.get("quant_prefix"))] # in the case that the quantitative columns have a prefix (like "Intensity " in MQ peptides.txt), only columns with the prefix are filtered
        else:
            return quantcolumn_candidates #in this case, we assume that all non-ionname/proteinname columns are quantitative columns


def get_ionname_columns(ion_dict, ion_hierarchy_local):
    ion_headers_merged = []
    ion_headers_grouped = []
    for lvl in ion_hierarchy_local:
        vals = ion_dict.get(lvl)
        ion_headers_merged.extend(vals)
        ion_headers_grouped.append(vals)
    return ion_headers_merged, ion_headers_grouped


def adapt_headers_on_extended_df(ion_headers_grouped, splitcol2sep):
    #in the case that one column has been split, we need to designate the "naming" column
    ion_headers_grouped_copy = copy.deepcopy(ion_headers_grouped)
    for vals in ion_headers_grouped_copy:
        if splitcol2sep is not None:
            for idx in range(len(vals)):
                if vals[idx] in splitcol2sep.keys():
                    vals[idx] = vals[idx] + "_idxs"
    return ion_headers_grouped_copy

def split_extend_df(input_df, splitcol2sep, value_threshold=10):
    """reformats data that is stored in a condensed way in a single column. For example isotope1_intensity;isotope2_intensity etc. in Spectronaut

    Args:
        input_df ([type]): [description]
        splitcol2sep ([type]): [description]
        value_threshold([type]): [description]

    Returns:
        Pandas Dataframe: Pandas dataframe with the condensed items expanded to long format
    """
    if splitcol2sep==None:
        return input_df

    for split_col, separator in splitcol2sep.items():
        idx_name = f"{split_col}_idxs"
        split_col_series = input_df[split_col].str.split(separator)
        input_df = input_df.drop(columns = [split_col])

        input_df[idx_name] = [list(range(len(x))) for x in split_col_series]
        exploded_input = input_df.explode(idx_name)
        exploded_split_col_series = split_col_series.explode()

        exploded_input[split_col] = exploded_split_col_series.replace('', 0) #the column with the intensities has to come after to column with the idxs

        exploded_input = exploded_input.astype({split_col: float})
        exploded_input = exploded_input[exploded_input[split_col]>value_threshold]
        #exploded_input = exploded_input.rename(columns = {'var1': split_col})
    return exploded_input



def add_merged_ionnames(df_subset, ion_hierarchy_local, ion_headers_grouped, quant_id_dict, hierarchy_type):
    all_ion_headers = list(itertools.chain.from_iterable(ion_headers_grouped))
    columns_to_index = [x for x in df_subset.columns if x not in all_ion_headers]
    df_subset = df_subset.set_index(columns_to_index)

    rows = df_subset[all_ion_headers].to_numpy()
    ions = []

    for row in rows: #iterate through dataframe
        count = 0
        ionstring = ""
        for lvl_idx in range(len(ion_hierarchy_local)):
            ionstring += f"{ion_hierarchy_local[lvl_idx]}"
            for sublvl in ion_headers_grouped[lvl_idx]:
                ionstring+= f"_{row[count]}_"
                count+=1
        ions.append(ionstring)
    df_subset['ion'] = ions
    df_subset = df_subset.reset_index()
    if quant_id_dict!= None:
        df_subset = df_subset.rename(columns = {quant_id_dict.get(hierarchy_type) : "quant_val"})
    return df_subset

In [ ]:
#export
def reformat_longtable_according_to_config_new(input_file, config_dict, conditions_subset = None, sep = "\t",decimal = "."):
    """Reshape a long format proteomics results table (e.g. Spectronaut or DIA-NN) to a wide format table.
    :param file input_file: long format proteomic results table
    :param string input_type: the configuration key stored in the config file (e.g. "diann_precursor")
    """
    relevant_cols = get_relevant_columns_config_dict(config_dict)
    input_df_it = pd.read_csv(input_file, sep = sep, decimal=decimal, usecols = relevant_cols, encoding ='latin1', chunksize=1000000)
    input_df_list = []
    for input_df_subset in input_df_it:
        input_df_subset = filter_input(config_dict.get("filters", {}), input_df_subset)
        if conditions_subset !=None:
            input_df_subset = input_df_subset[[(x in conditions_subset) for x in input_df_subset[config_dict.get("sample_id")]]]
            if len(input_df_subset.index)==0:
                continue
        if "ion_hierarchy" in config_dict.keys():
            input_df_subset = merge_protein_cols_and_ion_dict(input_df_subset, config_dict)
        else:
            input_df_subset = merge_protein_and_ion_cols(input_df_subset, config_dict)
        input_df_list.append(input_df_subset)


    input_df = pd.concat(input_df_list)

    input_df = input_df.astype({'quant_val': 'float'})
    input_reshaped = pd.pivot_table(input_df, index = ['protein', 'ion'], columns = config_dict.get("sample_ID"), values = 'quant_val', fill_value=0)
    if input_reshaped.iloc[:,0].replace(0, np.nan).median() <100: #when values are small, rescale by a constant factor to prevent rounding errors in the subsequent aq analyses
        input_reshaped = input_reshaped *10000

    input_reshaped = input_reshaped.reset_index()

    return input_reshaped

In [ ]:
#export
def read_wideformat_table(peptides_tsv, config_dict):
    input_df = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    filter_dict = config_dict.get("filters")
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df = filter_input(filter_dict, input_df)
    #input_df = merge_protein_and_ion_cols(input_df, config_dict)
    input_df = merge_protein_cols_and_ion_dict(input_df, config_dict)
    if 'quant_prefix' in config_dict.keys():
        quant_prefix = config_dict.get('quant_prefix')
        headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith(quant_prefix), input_df.columns))
        input_df = input_df[headers]
        input_df = input_df.rename(columns = lambda x : x.replace(quant_prefix, ""))

    input_df = input_df.reset_index()

    return input_df

In [ ]:

def read_mq_peptides_table(peptides_tsv, pepheader = "Sequence", protheader = "Leading razor protein"):
    peps = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    peps = peps[peps["Reverse"] != "+"]
    peps = peps[peps["Potential contaminant"] != "+"]
    if pepheader != None:
        peps = peps.rename(columns = {pepheader : "ion"})
    if protheader != None:
        peps = peps.rename(columns = {protheader: "protein"})
    headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith("Intensity "), peps.columns))
    peps = peps[headers]
    peps = peps.rename(columns = lambda x : x.replace("Intensity ", ""))

    return peps

In [ ]:
#export
from anytree.importer import JsonImporter
import os

def read_condpair_tree(cond1, cond2, results_folder = os.path.join(".", "results")):
    """reads the merged and clustered iontree for a given condpair"""
    condpairname = get_condpairname([cond1, cond2])
    tree_file =os.path.join(results_folder, f"{condpairname}.iontrees.json")
    if not os.path.isfile(tree_file):
        return None
    importer = JsonImporter()
    filehandle = open(tree_file, 'r')
    jsontree = importer.read(filehandle)
    filehandle.close()
    return jsontree

In [ ]:
#export

import numpy as np
import alphaquant.diffquant_utils as aqutils
import os

def merge_ptmsite_mappings_write_table(spectronaut_file, mapped_df, modification_type, ptm_type_config_dict = 'spectronaut_ptm_fragion_isotopes'):
    config_dict = aqutils.import_config_dict()
    config_dict_ptm = config_dict.get(ptm_type_config_dict)
    relevant_columns = aqutils.get_relevant_columns_config_dict(config_dict_ptm)
    specnaut_df_it = pd.read_csv(spectronaut_file, sep = "\t", chunksize=10000)
    ptmmapped_table_filename = f'{spectronaut_file.replace(".tsv", "")}_ptmsite_mapped.tsv'
    header = True
    for specnaut_df in specnaut_df_it:
        specnaut_df_annot = add_ptmsite_info_to_subtable(specnaut_df, mapped_df, modification_type, relevant_columns)
        write_chunk_to_file(specnaut_df_annot, ptmmapped_table_filename, header)
        header = False
    

def add_ptmsite_info_to_subtable(spectronaut_df, mapped_df, modification_type, relevant_columns):
    labelid2ptmid, labelid2site = get_ptmid_mappings(mapped_df) #get precursor+experiment to site mappings
    labelid_spectronaut = spectronaut_df["R.Label"].astype('str').to_numpy() + spectronaut_df["FG.Id"].astype('str').to_numpy() #derive the id to map from Spectronaut
    spectronaut_df["ptm_id"] = np.array([labelid2ptmid.get(x, np.nan) for x in labelid_spectronaut]) #add the ptm_id row to the spectronaut table
    modseq_typereplaced = np.array([str(x.replace(modification_type, "")) for x in spectronaut_df["EG.ModifiedSequence"]]) #EG.ModifiedSequence already determines a localization of the modification type. Replace all localizations and add the new localizations below
    sites = np.array([str(labelid2site.get(x)) for x in labelid_spectronaut])
    spectronaut_df["ptm_mapped_modseq"] = np.char.add(modseq_typereplaced, sites)
    spectronaut_df = spectronaut_df.dropna(subset=["ptm_id"]) #drop peptides that have no ptm
    spectronaut_df = spectronaut_df[relevant_columns]
    return spectronaut_df


def get_ptmid_mappings(mapped_df):
    labelid = mapped_df["R.Label"].astype('str').to_numpy() + mapped_df["FG.Id"].astype('str').to_numpy()
    ptm_ids = mapped_df["ptm_id"].to_numpy()
    site = mapped_df["site"].to_numpy()
    labelid2ptmid = dict(zip(labelid, ptm_ids))
    labelid2site = dict(zip(labelid, site))
    return labelid2ptmid, labelid2site


def write_chunk_to_file(chunk, filepath ,write_header):
    chunk.to_csv(filepath, header=write_header, mode='a', sep = "\t", index = None)

## check for already processed files

In [ ]:
#export
import os
def check_for_processed_runs_in_results_folder(results_folder):
    contained_condpairs = []
    folder_files = os.listdir(results_folder)
    result_files = list(filter(lambda x: "results.tsv" in x ,folder_files))
    for result_file in result_files:
        res_name = result_file.replace(".results.tsv", "")
        if ((f"{res_name}.normed.tsv" in folder_files) & (f"{res_name}.results.ions.tsv" in folder_files)):
            contained_condpairs.append(res_name)
    return contained_condpairs

## Wrapper functions

In [ ]:
#export
import pandas as pd
import os
import pathlib

def import_data(input_file, input_type_to_use = None, conditions_subset = None):
    """
    Function to import peptide level data. Depending on available columns in the provided file,
    the function identifies the type of input used (e.g. Spectronaut, MaxQuant, DIA-NN), reformats if necessary
    and returns a generic wide-format dataframe
    :param file input_file: quantified peptide/ion -level data
    :param file results_folder: the folder where the AlphaQuant outputs are stored
    """
    if "aq_reformat" in input_file:
        data = pd.read_csv(input_file, sep = "\t", encoding ='latin1')
        return data

    input_type, config_dict_type, sep = get_input_type_and_config_dict(input_file, input_type_to_use)
    format = config_dict_type.get('format')

    if format == "longtable":
        data = reformat_longtable_according_to_config_new(input_file, config_dict_type, conditions_subset, sep = sep)
    elif format == "widetable":
        data = read_wideformat_table(input_file, config_dict_type)
    else:
        raise Exception('Format not recognized!')

    if conditions_subset !=None:
        conditions_subset = sorted(conditions_subset)
        conditions_subset_string = "_".join(conditions_subset)
        input_type+=conditions_subset_string
    data.to_csv(f"{input_file}.{input_type}.aq_reformat.tsv",  index = False,sep = "\t")


    return data

In [ ]:
#export
import pandas as pd
import os.path
import pathlib

def get_input_type_and_config_dict(input_file, input_type_to_use = None):
    #parse the type of input (e.g. Spectronaut Fragion+MS1Iso) out of the input file


    config_file = os.path.join(pathlib.Path(__file__).parent.absolute(), "..", "intable_config.yaml") #the yaml config is located one directory below the python library files
    config_dict = load_config(config_file)
    type2relevant_columns = get_type2relevant_cols(config_dict)

    input_file = input_file.replace(".aq_reformat.tsv", "")

    filename = str(input_file)
    if '.csv' in filename:
        sep=','
    if '.tsv' in filename:
        sep='\t'
    if '.txt' in filename:
        sep='\t'

    if 'sep' not in locals():
        raise TypeError(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {input_file}! Your filename has to contain one of these extensions. Please modify your file name accordingly.")



    uploaded_data_columns = set(pd.read_csv(input_file, sep=sep, nrows=1, encoding ='latin1').columns)

    for input_type in type2relevant_columns.keys():
        if (input_type_to_use is not None) and (input_type!=input_type_to_use):
            continue
        relevant_columns = type2relevant_columns.get(input_type)
        relevant_columns = [x for x in relevant_columns if x] #filter None values
        print(f"recols {input_type}\t {relevant_columns}")
        if set(relevant_columns).issubset(uploaded_data_columns):
            config_dict_type =  config_dict.get(input_type)
            return input_type, config_dict_type, sep
    raise TypeError("format not specified in intable_config.yaml!")

In [ ]:
#export
def import_config_dict():
    config_file = os.path.join(pathlib.Path(__file__).parent.absolute(), "..", "intable_config.yaml") #the yaml config is located one directory below the python library files
    config_dict = load_config(config_file)
    return config_dict

In [ ]:
#export
import pandas as pd

def get_samplenames(data):
    """extracts the names of the samples of the AQ input dataframe"""
    names = list(data.columns)
    names.remove('protein')
    names.remove('ion')
    return names

In [ ]:
#export

import pandas as pd

def load_samplemap(samplemap_file):
    file_ext = os.path.splitext(samplemap_file)[-1]
    if file_ext=='.csv':
        sep=','
    if (file_ext=='.tsv') | (file_ext=='.txt'):
        sep='\t'

    if 'sep' not in locals():
        print(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {samplemap_file}! Trying with tab separation. In the case that it fails, please add the appropriate extension to your file name.")
        sep = "\t"

    return pd.read_csv(samplemap_file, sep = sep, encoding ='latin1')

In [ ]:
#export
def prepare_loaded_tables(data_df, samplemap_df):
    """
    Integrates information from the peptide/ion data and the samplemap, selects the relevant columns and log2 transforms intensities.
    """
    samplemap_df = samplemap_df[samplemap_df["condition"]!=""] #remove rows that have no condition entry
    filtvec_not_in_data = [(x in data_df.columns) for x in samplemap_df["sample"]] #remove samples that are not in the dataframe
    samplemap_df = samplemap_df[filtvec_not_in_data]
    headers = ['protein'] + samplemap_df["sample"].to_list()
    data_df = data_df.set_index("ion")
    for sample in samplemap_df["sample"]:
        data_df[sample] = np.log2(data_df[sample].replace(0, np.nan))
    return data_df[headers], samplemap_df

In [ ]:
#export

def import_acquisition_info_df(results_dir, last_ion_level_to_use = "CHARGE", header_filter = lambda x : ("EG." in x) | ("FG." in x), sep = "\t",decimal = "."):
    """import tables containing details on the acquisition (e.g. the Spectronaut input table)
    """
    method_params_dict = load_method_parameters(results_dir)
    input_file = method_params_dict.get('input_file')
    config_dict = get_input_type_and_config_dict(input_file)[1]

    df_sample = pd.read_csv(input_file, sep = sep, decimal = decimal, encoding='latin1', nrows=3000)

    df_sample = df_sample.replace({False: 0, True: 1})
    ion_headers, ion_levels = get_ion_headers_from_config_dict(config_dict, last_ion_level_to_use)
    
    numeric_headers =  list(df_sample.select_dtypes(include=np.number).columns)

    filtered_numeric_headers = [x for x in numeric_headers if header_filter(x)]
    header_subset = get_all_ion_headers(ion_headers) + filtered_numeric_headers

    input_df_it = pd.read_csv(input_file, sep = sep, decimal=decimal, usecols = header_subset, encoding ='latin1', chunksize=1000000)
    input_df_list = []
    for input_df_subset in input_df_it:
        input_df_subset = input_df_subset.drop_duplicates()

        input_df_list.append(input_df_subset)

    input_df = pd.concat(input_df_list)
    input_df = add_merged_ionnames(input_df, ion_levels, ion_headers, None, None)
    #input_df["ion"] = get_ion_row(input_df, ion_headers, ion_levels)


    return input_df

def get_ion_headers_from_config_dict(config_dict, last_ion_level_to_use):
    ion_headers = []
    ion_levels = []
    ion_hierarchy = config_dict.get("ion_hierarchy")
    hier_key = 'fragion' if 'fragion' in ion_hierarchy.keys() else list(ion_hierarchy.keys())[0]
    ion_hierarchy = ion_hierarchy.get(hier_key) #chose the first ion hierarchy object at random
    for idx in range(len(ion_hierarchy.get('order'))):
        ion_level = ion_hierarchy.get('order')[idx]
        ion_levels.append(ion_level)
        ion_headers.append(ion_hierarchy.get('mapping').get(ion_level))
        if ion_level == last_ion_level_to_use:
            break
    return ion_headers, ion_levels

def get_all_ion_headers(ion_headers):
    #ion headers is a dict of dict, merge all content of the ion headers
    all_ion_headers = []
    for ion_header in ion_headers:
        all_ion_headers.extend(ion_header)
    return all_ion_headers

def get_ion_row(df, ion_headers, ion_levels):
    all_ion_headers = get_all_ion_headers(ion_headers)
    df_subset_np = df[all_ion_headers].to_numpy()
    ionnames = [get_ion_header(row, ion_headers, ion_levels) for row in df_subset_np]
    return ionnames

def get_ion_header(df_row_np, ion_headers ,ion_levels):
    ion = ""
    idx_column = 0
    for idx_level in range(len(ion_levels)):
        level = str(ion_levels[idx_level]) + "_"
        name = ""
        for idx_sublevel in range(len(ion_headers[idx_level])): #in the case that multiple columns determine one level, e.g. the level "FRAGION" is determined by the columns for fragment ion, losses, and fragion charge state
            name += str(df_row_np[idx_column]) + "_"
            idx_column+=1
        ion+=level + name
    return ion

In [ ]:
#export

def merge_acquisition_df_parameter_df(acquisition_df, parameter_df, groupby_merge_type = 'mean'):
    """acquisition df contains details on the acquisition, parameter df are the parameters derived from the tree
    """
    merged_df = parameter_df.merge(acquisition_df, how = 'left', on = 'ion')
    if groupby_merge_type == 'mean':
        merged_df = merged_df.groupby('ion').mean().reset_index()
    if groupby_merge_type == 'min':
        merged_df = merged_df.groupby('ion').min().reset_index()
    if groupby_merge_type == 'max':
        merged_df = merged_df.groupby('ion').max().reset_index()
    merged_df = merged_df.dropna(axis=1, how='all')
    return merged_df